In [1]:
import os
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"] = ".80"

import numpy as np

import jax
import jax.numpy as jnp
import spyx
import haiku as hk
import optax
from jax_tqdm import scan_tqdm

### SHD Dataloading

In [2]:

import tonic
from tonic import datasets, transforms
import torchvision as tv
from torch.utils.data import DataLoader, Subset
from collections import namedtuple

State = namedtuple("State", "obs labels")



In [4]:
sensor_size = tonic.datasets.NMNIST.sensor_size

# Denoise removes isolated, one-off events
# time_window
frame_transform = transforms.Compose([
                                      transforms.ToFrame(sensor_size=sensor_size, 
                                                         n_time_bins=64),
                                      lambda x: np.packbits(x, axis=0)
                                     ])

train_dataset = tonic.datasets.NMNIST(save_to='./tmp/data', first_saccade_only=True, transform=frame_transform, train=True)
test_dataset = tonic.datasets.NMNIST(save_to='./tmp/data', first_saccade_only=True, transform=frame_transform, train=False)

In [5]:
train_dl = iter(DataLoader(train_dataset, batch_size=len(train_dataset),
                          collate_fn=tonic.collation.PadTensors(batch_first=True), drop_last=True, shuffle=False))
        
x_train, y_train = next(train_dl)

In [6]:
test_dl = iter(DataLoader(test_dataset, batch_size=len(test_dataset),
                          collate_fn=tonic.collation.PadTensors(batch_first=True), drop_last=True, shuffle=False))
        
x_test, y_test = next(test_dl)

In [8]:
x_train = jnp.array(x_train, dtype=jnp.uint8)
y_train = jnp.array(y_train, dtype=jnp.uint8)

x_test = jnp.array(x_test, dtype=jnp.uint8)
y_test = jnp.array(y_test, dtype=jnp.uint8)

In [9]:
def shuffle(dataset, shuffle_rng):
    x, y = dataset

    cutoff = y.shape[0] % batch_size

    obs = jax.random.permutation(shuffle_rng, x, axis=0)[:-cutoff]
    labels = jax.random.permutation(shuffle_rng, y, axis=0)[:-cutoff]

    obs = jnp.reshape(obs, (-1, batch_size) + obs.shape[1:])
    labels = jnp.reshape(labels, (-1, batch_size)) # should make batch size a global

    return State(obs=obs, labels=labels)

### Spyx SHD

In [12]:
def shd_snn(x): 
        
    core = hk.DeepRNN([
        hk.Conv2d(2, 12, 5),
        spyx.nn.LIF((64,), activation=spyx.axn.Axon(spyx.axn.arctan())),
        hk.MaxPool2d(2),
        hk.Conv2d(12, 32, 5),
        spyx.nn.LIF((64,), activation=spyx.axn.Axon(spyx.axn.arctan())),
        hk.MaxPool2d(),
        hk.Flatten(),
        hk.Linear(20, with_bias=False),
        spyx.nn.LIF((20,), activation=spyx.axn.Axon(spyx.axn.arctan()))
    ])
    
    # static unroll for maximum performance
    spikes, V = hk.dynamic_unroll(core, x, core.initial_state(x.shape[0]), time_major=False, unroll=32)
    
    return spikes, V

In [13]:
key = jax.random.PRNGKey(0)
# Since there's nothing stochastic about the network, we can avoid using an RNG as a param!
sample_x, sample_y = shuffle((x_train,y_train),key)
SNN = hk.without_apply_rng(hk.transform(shd_snn))
params = SNN.init(rng=key, x=jnp.float32(sample_x[0]))

In [14]:
def gd(SNN, params, dataset, epochs=300):
        
    opt = optax.adam(learning_rate=5e-4)
    
    # create and initialize the optimizer
    opt_state = opt.init(params)
    grad_params = params
        
    # define and compile our eval function that computes the loss for our SNN
    @jax.jit
    def net_eval(weights, events, targets):
        readout = SNN.apply(weights, events)
        traces, V_f = readout
        return spyx.fn.integral_crossentropy(traces, targets) # smoothing needs to be more explicit in docs...
        
    # Use JAX to create a function that calculates the loss and the gradient!
    surrogate_grad = jax.value_and_grad(net_eval) 
        
    rng = jax.random.PRNGKey(0)        
    
    # compile the meat of our training loop for speed
    @jax.jit
    def train_step(state, data):
        grad_params, opt_state = state
        events, targets = data # fix this
        events = jnp.unpackbits(events, axis=1) # decompress temporal axis
        # compute loss and gradient
        loss, grads = surrogate_grad(grad_params, events, targets)
        # generate updates based on the gradients and optimizer
        updates, opt_state = opt.update(grads, opt_state, grad_params)
        # return the updated parameters
        new_state = [optax.apply_updates(grad_params, updates), opt_state]
        return new_state, loss
    
    
    # Here's the start of our training loop!
    @scan_tqdm(epochs)
    def epoch(epoch_state, epoch_num):
        curr_params, curr_opt_state = epoch_state

        shuffle_rng = jax.random.fold_in(rng, epoch_num)
        train_data = shuffle(dataset, shuffle_rng)
        
        # train epoch
        end_state, train_loss = jax.lax.scan(
            train_step,# func
            [curr_params, curr_opt_state],# init
            train_data,# xs
            train_data.obs.shape[0]# len
        )
                    
        return end_state, jnp.mean(train_loss)
    # end epoch
    
    # epoch loop
    final_state, metrics = jax.lax.scan(
        epoch,
        [grad_params, opt_state], # metric arrays
        jnp.arange(epochs), # 
        epochs # len of loop
    )
    
    final_params, _ = final_state
    
                
    # return our final, optimized network.       
    return final_params, metrics

In [15]:
def test_gd(SNN, params, dataset):

    @jax.jit
    def test_step(params, data):
        events, targets = data
        events = jnp.unpackbits(events, axis=1)
        readout = SNN.apply(params, events)
        traces, V_f = readout
        acc, pred = spyx.fn.integral_accuracy(traces, targets)
        loss = spyx.fn.integral_crossentropy(traces, targets)
        return params, [acc, loss, pred, targets]
    
    test_data = shuffle(dataset, jax.random.PRNGKey(0))
    
    _, test_metrics = jax.lax.scan(
            test_step,# func
            params,# init
            test_data,# xs
            test_data.obs.shape[0]# len
    )
    
    acc = jnp.mean(test_metrics[0])
    loss = jnp.mean(test_metrics[1])
    preds = jnp.array(test_metrics[2]).flatten()
    tgts = jnp.array(test_metrics[3]).flatten()
    return acc, loss, preds, tgts

In [16]:
grad_params, metrics = gd(SNN, params, (x_train,y_train), epochs=300)


  0%|          | 0/300 [00:00<?, ?it/s]

In [17]:
metrics

Array([3.3948634, 3.0212855, 3.03532  , 3.023731 , 2.9759493, 2.8594928,
       2.7836218, 2.7019773, 2.6405947, 2.5995522, 2.5443957, 2.5140488,
       2.47452  , 2.4482877, 2.4146333, 2.376006 , 2.3507137, 2.3295076,
       2.3165836, 2.2969234, 2.2816613, 2.268716 , 2.2536645, 2.227889 ,
       2.2070014, 2.1970618, 2.1781995, 2.160053 , 2.135837 , 2.123934 ,
       2.1015546, 2.0945477, 2.07775  , 2.0695205, 2.0600803, 2.0559309,
       2.0453296, 2.0422237, 2.031036 , 2.0284038, 2.018376 , 2.0102658,
       2.006234 , 2.008062 , 1.9893986, 1.9828401, 1.9807476, 1.982857 ,
       1.9694746, 1.9630796, 1.967521 , 1.9541516, 1.9514204, 1.9487829,
       1.941763 , 1.9421433, 1.9427478, 1.9334282, 1.9261316, 1.9253156,
       1.9206196, 1.9161831, 1.9150175, 1.9136933, 1.9085118, 1.9031168,
       1.9061825, 1.8991681, 1.8885156, 1.8890601, 1.8875759, 1.8813595,
       1.8773499, 1.8780856, 1.8788662, 1.8749352, 1.8716233, 1.868697 ,
       1.8597573, 1.8643638, 1.8605853, 1.8526983, 

In [18]:
acc, loss, preds, tgts = test_gd(SNN, grad_params, (x_test,y_test))
print("Accuracy:", acc, "Loss:", loss)

Accuracy: 0.74560547 Loss: 1.9869497
